<a href="https://colab.research.google.com/github/rajlm10/D2L-Torch/blob/main/D2L_LNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils import data
import numpy as np

In [2]:
def generate_data(w,b,samples):
  """Generate y = Xw + b +  Gaussian noise."""
  X=torch.normal(0,1,(samples,len(w)))
  y=torch.matmul(X,w)+ b
  y+=torch.normal(0,0.01,y.shape)
  return X, y.reshape(-1,1)


In [4]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = generate_data(true_w, true_b, 1000)

In [22]:
def create_iterator(dataset,batch_size,is_train=True):
  dataset=data.TensorDataset(*dataset)

  return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [24]:
data_iter=create_iterator((features,labels),10,True)
next(iter(data_iter))

[tensor([[-0.2900,  1.5004],
         [-0.5602,  0.1621],
         [ 0.6776,  0.1689],
         [ 0.2050, -0.4512],
         [ 1.5926,  2.4947],
         [ 1.0554,  0.6057],
         [-0.9335, -0.0710],
         [-1.1466,  0.0096],
         [-0.0272,  0.0818],
         [ 0.7346,  0.0961]]), tensor([[-1.4670],
         [ 2.5116],
         [ 4.9676],
         [ 6.1549],
         [-1.0749],
         [ 4.2580],
         [ 2.5624],
         [ 1.8775],
         [ 3.8652],
         [ 5.3483]])]

In [25]:
data_iter

In [27]:
features.shape,labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [28]:
from torch import nn

net=nn.Sequential(nn.Linear(features.shape[1],labels.shape[1]))

In [29]:
#Initialize model params
net[0]

Linear(in_features=2, out_features=1, bias=True)

In [31]:
net[0].weight,net[0].bias

(Parameter containing:
 tensor([[0.1584, 0.0882]], requires_grad=True), Parameter containing:
 tensor([0.0942], requires_grad=True))

In [32]:
net[0].weight.data.normal_(0, 0.01) 
net[0].bias.data.fill_(0)
net[0].weight,net[0].bias

(Parameter containing:
 tensor([[-0.0020,  0.0064]], requires_grad=True), Parameter containing:
 tensor([0.], requires_grad=True))

In [33]:
net.parameters()

<generator object Module.parameters at 0x7f2071ae05d0>

In [34]:
loss=nn.MSELoss()

trainer=torch.optim.SGD(net.parameters(),lr=0.03)

In [36]:
net(next(iter(data_iter))[0]).shape,next(iter(data_iter))[1].shape

(torch.Size([10, 1]), torch.Size([10, 1]))

In [37]:
loss(net(next(iter(data_iter))[0]),next(iter(data_iter))[1])

tensor(17.2331, grad_fn=<MseLossBackward0>)

In [38]:
for epoch in range(3):
  for X,y in data_iter:
    l=loss(net(X),y) # yhat,y_true
    trainer.zero_grad()
    l.backward()
    trainer.step()

  l=loss(net(features),labels)
  print(f'''Loss for epoch {epoch+1} is {l}''')

Loss for epoch 1 is 0.0002844185510184616
Loss for epoch 2 is 9.692551248008385e-05
Loss for epoch 3 is 9.706428681965917e-05
